# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra # pip install cassandra-driver
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/kevin/Documents/practice/udacity/sparkify2/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Cassandra Queries. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## The following are 3 queries for three questions of the data

### 1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## Query 1:  Get the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS events_by_sessionid_iteminsession "
query1 = query1 + "(artist_name text, song_title text, duration decimal, session_id int, item_num_in_session int, PRIMARY KEY ((session_id, item_num_in_session), artist_name, song_title))"

try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO events_by_sessionid_iteminsession (artist_name, song_title, duration, session_id, item_num_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query1_2 = "SELECT artist_name, song_title, duration FROM events_by_sessionid_iteminsession WHERE session_id=338 AND item_num_in_session=4"
try:
    rows = session.execute(query1_2)
except Exception as e:
    print(e)
    
data = []
for row in rows:
    data.append((row.artist_name, row.song_title, float(row.duration)))
    
df1 = pd.DataFrame.from_records(data, columns=['artist_name', 'song_title', 'song length'])
df1.head()
# print(data)

,artist_name,song_title,song length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### 2: Get the name of artists and songs (sorted by ItemInSession) and first and last name of user for user_id=10 and sessionid=182

In [11]:
# Get the name of artists and songs (sorted by ItemInSession) and first and last name of user for user_id=10 and sessionid=182
query2 = "CREATE TABLE IF NOT EXISTS artist_song_user_by_sessionid_iteminsession "
query2 = query2 + "(artist_name text, song_title text, session_id int, item_num_in_session int, first_name text, last_name text, user_id int, PRIMARY KEY ((user_id, session_id), item_num_in_session, artist_name, song_title))"

try:
    session.execute(query2)
except Exception as e:
    print(e)
                  

                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_user_by_sessionid_iteminsession (artist_name, song_title, session_id, item_num_in_session, first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], int(line[8]), int(line[3]), line[1], line[4], int(line[10])))

Verify that the data have been inserted into each table

In [13]:
## Add in the SELECT statement to verify the data was entered into the table
query1_2 = "SELECT artist_name, song_title, first_name, last_name FROM artist_song_user_by_sessionid_iteminsession WHERE user_id=10 AND session_id=182 ALLOW FILTERING"
try:
    rows = session.execute(query1_2)
except Exception as e:
    print(e)

data2 = []
for row in rows:
    data2.append((row.artist_name, row.song_title, row.first_name, row.last_name))
    
df2 = pd.DataFrame.from_records(data2, columns=['artist_name', 'song_title', 'first_name', 'last_name'])
df2.head()

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3: Get user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
## Query 3: Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS users_by_song_id"
query3 = query3 + "(first_name text, last_name text, song_title text, PRIMARY KEY (song_title, first_name, last_name))"

try:
    session.execute(query3)
except Exception as e:
    print(e)
              

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_song_id (first_name, last_name, song_title)"
        query = query + " VALUES (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[1], line[4], line[9]))

In [16]:
## Add in the SELECT statement to verify the data was entered into the table
query1_3 = "SELECT first_name, last_name FROM users_by_song_id WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query1_3)
except Exception as e:
    print(e)
    
data3 = []
for row in rows:
    data3.append((row.first_name, row.last_name))
    
df3 = pd.DataFrame.from_records(data3, columns=['first_name', 'last_name'])
df3.head()

,first_name,last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

In [18]:
drop_queries = [
    "DROP TABLE IF EXISTS events_by_sessionid_iteminsession",
    "DROP TABLE IF EXISTS artist_song_user_by_sessionid_iteminsession",
    "DROP TABLE IF EXISTS users_by_song_id"
]
for drop_query in drop_queries:
    try:
        rows = session.execute(drop_query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()